In [1]:
%load_ext autoreload
%autoreload 2
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import os
# from keras.utils.np_utils import to_categorical
import numpy as np
from agents.knn import *
from utils import generate_id
from settings import settings

# def reduce_actions(actions):
#     s = np.unique(np.array([a['steer'] for a in actions]))
#     s.sort()
#     steer_values = np.unique(np.concatenate((
#         np.linspace(-1, -0.1, 15),
#         np.linspace(-0.1, 0 , 6),
#         np.linspace(0, 0.1, 6),
#         np.linspace(0.1, 1, 15),
#     )))
#     return [
#         {
#             **a,
#             'steer': steer_values[
#                 np.argmin(
#                     np.abs(
#                         steer_values - a['steer']
#                     )
#                  )
#             ]
#         } 
#         for a in actions
#     ]

In [2]:
data = parse_dataset(
    '/home/kku/Dropbox/datasets/cg_track_2_2'
#     '/home/kku/Dropbox/important_logs',
#     8
)
test_size = 0.10

action_keys = [
    'steer',
    'accel',
    'brake'
]

state_keys = [
    'speedX',
    'speedY',
    'speedZ',
    'angle',
    'trackPos',
    'distFromStart',
    'track',
    'wheelSpinVel'
]

states, actions = extract_parameters(
    data,
    action_keys=action_keys,
    state_keys=state_keys
)
data_actions, actions = unify_actions(actions)
X, Y = sklearn.utils.shuffle(np.array(states), np.array(data_actions))
print(X.shape, Y.shape, np.unique(Y).shape)

(78138, 29) (78138,) (243,)


In [3]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=test_size)
print(X_train.shape)

(70324, 29)


In [10]:
scaler = sklearn.preprocessing.MinMaxScaler((-1,1))  
scaler.fit(X_train)  

classifier = sklearn.ensemble.ExtraTreesClassifier(20)
# classifier = sklearn.tree.DecisionTreeClassifier(min_samples_leaf=4)

classifier.fit(
    scaler.transform(X_train),
    Y_train
) 

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [ ]:
print(
    'Traning score: ', classifier.score(scaler.transform(X_train), Y_train), '\n'
    'Test score: ', classifier.score(scaler.transform(X_test), Y_test)
)
# sklearn.metrics.confusion_matrix(Y_test, classifier.predict(X_test))

Traning score:  0.999445424037313 
Test score:  0.6584335807524955


In [ ]:
def dump(path = 'saved_model'):
    print(f'Dumping model to: {path}')
    os.system(f'mkdir {path}')
    np.savez(
        f'{path}/parameters.npz',
        **{
            'actions': actions,
            'state_keys': state_keys
        }
    
    )
    joblib.dump(classifier, f'{path}/classifier')
    joblib.dump(scaler, f'{path}/scaler')    
path=f'models/{generate_id()}'
def remove_dump():
    print(os.system(f'rm -r {path}'))
dump(path)
    

Dumping model to: models/2019-01-05_21:32:08_252


In [25]:
asd = X_train[0]
asd.shape

(29,)

In [26]:
scaler.transform([asd])

array([[-1.0927977 ,  0.30305797, -0.00175801,  0.16198501,  0.21558777,
         1.46309705, -0.20766877, -0.33272769, -0.50611525, -0.25591718,
         0.49272732,  1.35532288,  1.41963225,  1.32282782,  1.27475804,
         1.25183432,  1.24446858,  1.25392829,  1.29899214,  0.95950986,
         0.38272102,  0.02796284,  0.07798637,  0.24934772,  0.19622016,
        -0.9676149 , -0.95126154, -1.12915632, -1.11655785]])

In [7]:
remove_dump()

0
